# Tactile Sensor Example

In [ ]:
import airo_models
import numpy as np
from airo_models.primitives.sphere import sphere_geometry_dict

def sensor_urdf_dict():

    base_link = {"@name": "base_link"}

    links = [base_link]
    joints = []

    # add a grid of spheres as links to the sensor
    sphere_size = 0.01 # diameter
    sphere_radius = sphere_size / 2

    # sensor is a 2x8x4 grid of spheres
    x_centers = np.linspace(-0.005, 0.005, 2) 
    y_centers = np.linspace(-0.035, 0.035, 8)
    z_centers = np.linspace(-0.015, 0.015, 4)

    sphere_geometry = sphere_geometry_dict(radius=sphere_radius)

    for i, x in enumerate(x_centers):
        for j, y in enumerate(y_centers):
            for k, z in enumerate(z_centers):

                tixel_link = {
                    "@name": f"tixel_{i}_{j}_{k}",
                    "visual": {
                        "geometry": sphere_geometry,
                    },
                    "collision": {
                        "geometry": sphere_geometry,
                    }
                }

                joint = {
                    "@name": f"joint_{i}_{j}_{k}",
                    "@type": "fixed",
                    "parent": { "@link": "base_link" },
                    "child": { "@link": f"tixel_{i}_{j}_{k}" },
                    "origin": {
                        "@xyz": f"{x} {y} {z}"
                    }
                }

                links.append(tixel_link)
                joints.append(joint)
                
    urdf_dict = {
        "robot": {
            "@name": "sensor",
            "link": links,
            "joint": joints
        }
    }

    return urdf_dict

sensor_urdf_dict_ = sensor_urdf_dict()
sensor_urdf_path = airo_models.urdf.write_urdf_to_tempfile(sensor_urdf_dict_, prefix="tactile_sensor_")
sensor_urdf_path

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import add_manipulator, add_floor, add_meshcat, finish_build
from airo_drake import SingleArmScene
from pydrake.math import RigidTransform


robot_diagram_builder = RobotDiagramBuilder() 

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur5e", "robotiq_2f_85", static_gripper=True)
add_floor(robot_diagram_builder)


# adding sensor area bbox
plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()

sensor_transform = RigidTransform(p=np.array([0, 0, 0.13]))
sensor_index = parser.AddModels(sensor_urdf_path)[0]
sensor_frame = plant.GetFrameByName("base_link", sensor_index)

# welding the sensor to the gripper
gripper_frame = plant.GetFrameByName("base_link", gripper_index)
plant.WeldFrames(gripper_frame, sensor_frame, sensor_transform)


# adding a thin (x) obstable

obstacle_urdf_path = airo_models.box_urdf_path((0.01, 0.5, 0.25), "obstacle")
obstacle_transform = RigidTransform(p=np.array([0.3, 0, 0.15]))
obstacle_index = parser.AddModels(obstacle_urdf_path)[0]
obstacle_frame = plant.GetFrameByName("base_link", obstacle_index)
plant.WeldFrames(plant.world_frame(), obstacle_frame, obstacle_transform)



robot_diagram, context = finish_build(robot_diagram_builder, meshcat)
del robot_diagram_builder # no longer needed

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)
scene

In [ ]:
from airo_drake import visualize_frame

plant_context = plant.GetMyContextFromRoot(context)

gripper_body = plant.GetBodyByName("base_link", gripper_index)
gripper_pose = plant.EvalBodyPoseInWorld(plant_context, gripper_body)

visualize_frame(scene.meshcat, "tcp_frame", gripper_pose)

In [ ]:
plant = scene.robot_diagram.plant()
joints_1 = np.deg2rad([0, -64, -120, -90, 90, 90])

plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, scene.arm_index, joints_1)
scene.robot_diagram.ForcedPublish(context) # updates the meshcat visualization

In [ ]:
from pydrake.planning import SceneGraphCollisionChecker

collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index, sensor_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.0,
    self_collision_padding=0.0,
)

In [ ]:
from airo_drake import list_collisions_between_bodies

def print_robot_collisions(q):
    colliding_bodies = list_collisions_between_bodies(collision_checker, q)

    for colliding_body_1, colliding_body_2, is_self_collision in colliding_bodies:
            bodyA_name = plant.get_body(colliding_body_1).scoped_name()
            bodyB_name = plant.get_body(colliding_body_2).scoped_name()
            print(f"Body {bodyA_name} is colliding with body {bodyB_name} (is self collision: {is_self_collision})")

print_robot_collisions(joints_1)

In [ ]:
def read_sensor(q): 
    colliding_bodies = list_collisions_between_bodies(collision_checker, q)
    readout = np.zeros((2, 8, 4))

    for colliding_body_1, colliding_body_2, is_self_collision in colliding_bodies:
            bodyA_name = plant.get_body(colliding_body_1).scoped_name()
            # bodyB_name = plant.get_body(colliding_body_2).scoped_name()

            if "tixel_" not in str(bodyA_name): # TODO check if this assumption is OK
                    continue
            
            i, j, k = [int(x) for x in str(bodyA_name).split("_")[1:]]
            readout[i, j, k] = 1

    return readout
            
            
readout = read_sensor(joints_1)
readout

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create an empty color array with the same shape as readout
colors = np.empty((8,8,8), dtype=object)

for i in range(2):
    for j in range(8):
        for k in range(4):
            if readout[i, j, k]:
                colors[i, j, k] = 'red'
            else:
                colors[i, j, k] = 'green'

# and plot everything
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(colors, facecolors=colors, edgecolor='k', alpha=0.9)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

readout = read_sensor(joints_1)
plt.imshow(readout.sum(axis=0))
# label axes
plt.xlabel("X")
plt.ylabel("Y")
plt.show()